## Activation and Custom Layers

## Imports

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Layer
from tensorflow.keras import mixed_precision

## Adding an activation layer

To use the built-in activations in Keras, we can specify an `activation` parameter in the `__init__()` method of our custom layer class. From there, we can initialize it by using the `tf.keras.activations.get()` method. This takes in a string identifier that corresponds to one of the [available activations](https://keras.io/api/layers/activations/#available-activations) in Keras.

In [2]:
class SimpleDense(Layer): ## sub class of keras.layers.Layers

    # add an activation parameter
    def __init__(self, units=32, activation=None):
        '''
        Here the the initial parameters of the layer are defined.
        Note that we still need to call the super init function
        '''
        super(SimpleDense, self).__init__()
        self.units = units
        self.activation = tf.keras.activations.get(activation)
        
    def build(self, input_shape):
        '''
        This function initialises the weights and biases.
        input_shape is expected when the fit method is called.
        '''
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(name = 'weights',
                             initial_value = w_init(shape=(input_shape[-1], self.units),
                                                    dtype='float32'),
                            trainable = True)
        
        b_init = tf.random_normal_initializer()
        self.b = tf.Variable(name = 'bias',
                             initial_value = b_init(shape=(self.units,), dtype='float32'),
                            trainable = True)
        super().build(input_shape) ## calls build function of Layer class once everything is defined
        
        #another way of initialising weights
        '''
              self.w = self.add_weight(shape=(input_shape[-1], self.units),
                               initializer='random_normal',
                               trainable=True)
                               
              self.b = self.add_weight(shape=(self.units,),
                               initializer='random_normal',
                               trainable=True)

        '''

    def call(self, inputs):
        """
        How the computation happens in the layer and what is returned at the end.
        inputs are expectedwhen the layer is called.
        """
        matmul = tf.matmul(inputs, self.w) + self.b
        return self.activation(matmul)

The string identifier is mostly the same as the function name so 'relu' below will get `tf.keras.activations.relu`.

In [3]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    SimpleDense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)
model.evaluate(x_test, y_test)

Train on 60000 samples
Epoch 1/5
60000/60000 [==============================] - 5s 83us/sample - loss: 0.2965 - accuracy: 0.9141
Epoch 2/5
60000/60000 [==============================] - 5s 78us/sample - loss: 0.1451 - accuracy: 0.9575
Epoch 3/5
60000/60000 [==============================] - 5s 78us/sample - loss: 0.1058 - accuracy: 0.9685
Epoch 4/5
60000/60000 [==============================] - 5s 77us/sample - loss: 0.0879 - accuracy: 0.9726
Epoch 5/5
10000/10000 [==============================] - 0s 29us/sample - loss: 0.0751 - accuracy: 0.9777


[0.07513838542420417, 0.9777]